# Df 0 1 6_Vec Ops
This tutorial shows the potential of the VecOps approach for treating collections
stored in datasets, a situation very common in HEP data analysis.




**Author:** Danilo Piparo  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 01:07 AM.</small></i>

In [ ]:
using ROOT::RDataFrame;
using namespace ROOT::VecOps;

We re-create a set of points in a square.
 This is a technical detail, just to create a dataset to play with!

In [ ]:
auto unifGen = [](double) { return gRandom->Uniform(-1.0, 1.0); };
auto vGen = [&](int len) {
   RVec<double> v(len);
   std::transform(v.begin(), v.end(), v.begin(), unifGen);
   return v;
};
RDataFrame d(1024);
auto d0 = d.Define("len", []() { return (int)gRandom->Uniform(0, 16); })
   .Define("x", vGen, {"len"})
   .Define("y", vGen, {"len"});

Now we have in hands d, a rdataframe with two columns, x and y, which
 hold collections of coordinates. The size of these collections vary.
 Let's now define radii out of x and y. We'll do it treating the collections
 stored in the columns without looping on the individual elements.

In [ ]:
auto d1 = d0.Define("r", "sqrt(x*x + y*y)");

Now we want to plot 2 quarters of a ring with radii .5 and 1
 Note how the cuts are performed on RVecs, comparing them with integers and
 among themselves

In [ ]:
auto ring_h = d1.Define("rInFig", "r > .4 && r < .8 && x*y < 0")
                 .Define("yFig", "y[rInFig]")
                 .Define("xFig", "x[rInFig]")
                 .Histo2D({"fig", "Two quarters of a ring", 64, -1, 1, 64, -1, 1}, "xFig", "yFig");

auto cring = new TCanvas();
ring_h->DrawCopy("Colz");

return 0;

Draw all canvases 

In [ ]:
gROOT->GetListOfCanvases()->Draw()